In [18]:
# imports
import shutil
from pathlib import Path

from ipywidgets import IntProgress, Label, VBox
from IPython.display import display

In [19]:
# set network path and get list of directories to process
data_directory = Path('/Users/jeremy/Pictures/KJA/Output/1963')
input_directory = data_directory.joinpath('1.toOCR')
output_directory = data_directory.joinpath('2.OCRed')

directories_to_convert_to_pdf_paths_list = sorted([x for x in input_directory.iterdir() if x.is_dir()])

print(f'{len(directories_to_convert_to_pdf_paths_list)} directories to process')

10 directories to process


In [20]:
# functions

def get_tif_paths_list(directory_path, scantailor=False):
    
    # if directory_path not a Path-like object then make it one
    if not isinstance(directory_path, Path):
        directory_path = Path(directory_path)
        
    # add 'out' path to directory if processed with ScanTailor
    if scantailor:
        directory_path = Path(directory_path).joinpath('out')
    
    # get paths to *.tif to process
    tif_paths_list = sorted(directory_path.glob('*.tif'))
    
    if len(tif_paths_list) == 0:
        tif_paths_list = sorted(directory_path.glob('*.tiff'))
    
    # only keep *.tif that DO NOT start with '.' to exclude macOS index files
    tif_paths_list = [x for x in tif_paths_list if not str(x.name).startswith('.')]
    
    return tif_paths_list


def create_temporary_pdf_directory(directory_path):
    
    # if directory_path not a Path-like object then make it one
    if not isinstance(directory_path, Path):
        directory_path = Path(directory_path)
        
    temporary_pdf_directory_path = Path.home().joinpath(f'Desktop/_temporary_pdfs/_temp_{directory_path.name}')
    
    try:  # creating the temporary directory
        temporary_pdf_directory_path.mkdir(parents=True)
    except FileExistsError:  # delete the directory & all contents then create it
        shutil.rmtree(temporary_pdf_directory_path)
        temporary_pdf_directory_path.mkdir()
        
    return temporary_pdf_directory_path


def batch_ocr_tifs_into_pdfs(tif_paths_list, temporary_pdf_directory_path):
    
    number_of_tifs = len(tif_paths_list)
    number_of_pdfs = 0
    
    # progress bar
    progress_label = Label('OCR *.tif')
    progress_bar = IntProgress(min=0, max=number_of_tifs)
    progress_widget = VBox([progress_label, progress_bar])
    display(progress_widget)
    
    for tif_path in tif_paths_list:
        
        label = f'OCRing {tif_path.name}'
        progress_label.value = label

        temporary_pdf_output_path = temporary_pdf_directory_path.joinpath(tif_path.stem)

        # ocr *.tif with tesseract
        # NOTE: output SHOULD NOT have extension on end!
        !tesseract "{str(tif_path)}" "{str(temporary_pdf_output_path)}" pdf 2>/dev/null

        # Add the suffix .pdf that tesseract didn't want
        temporary_pdf_output_path = temporary_pdf_directory_path.joinpath(f'{tif_path.stem}.pdf') 

        if temporary_pdf_output_path.is_file():
            number_of_pdfs += 1
            progress_bar.value = number_of_pdfs
        
    if number_of_pdfs != number_of_tifs:
        print(f'ERROR - ERROR - ERROR')
        print(f'# of *.tif: {number_of_tifs}')
        print(f'# of PDFs: {number_of_pdfs}')
    else:
        print(f'{number_of_pdfs} PDFs created and saved in {temporary_pdf_directory_path}')
    
    
def concatenate_pdfs(temporary_pdf_directory_path):

    # strip '_temporary_pdfs_' from beginning of directory path to get combined PDF name
    combined_pdf_name = f"{str(temporary_pdf_directory_path).split('_temp_')[1]}.pdf"

    combined_pdf_output_path = temporary_pdf_directory_path.joinpath(combined_pdf_name)

    # concatenate with pdftk
    !pdftk "{str(temporary_pdf_directory_path)}"/*.pdf cat output "{str(combined_pdf_output_path)}"

    if combined_pdf_output_path.is_file() and combined_pdf_output_path.stat().st_size > 0:
        print(f'{combined_pdf_output_path} created with size {round((combined_pdf_output_path.stat().st_size/1024/1024/1024), 2)} GB')

    return combined_pdf_output_path

In [21]:
for dir_path in directories_to_convert_to_pdf_paths_list:
    
    # get list of *.tif to process
    tif_paths_list = get_tif_paths_list(dir_path)
    
    number_of_tifs = len(tif_paths_list)
    print(f'{number_of_tifs} *.tif to process in {dir_path}')
    
    # create temp directory
    temp_pdf_dir_path = create_temporary_pdf_directory(dir_path)
    
    # OCR tifs into PDFs with progress bar
    batch_ocr_tifs_into_pdfs(tif_paths_list, temp_pdf_dir_path)
    
    # concatenate PDFs into single file
    combined_pdf_path = concatenate_pdfs(temp_pdf_dir_path)
    
    # copy combined PDF to copy directory
    output_pdf_path = output_directory.joinpath(combined_pdf_path.name)
    shutil.copy(combined_pdf_path, output_pdf_path)
    
    if output_pdf_path.is_file():  # move processed dir_path into output_directory
        renamed_dir_path = output_directory.joinpath(dir_path.name)
        dir_path.rename(renamed_dir_path)

    print('')
    print('*****')    
    print(f'{dir_path.name} is now {renamed_dir_path}')
    print('*****')
    print('')

4 *.tif to process in /Users/jeremy/Pictures/KJA/Output/1963/1.toOCR/KJA_RG-D_1a-21_1963-01_v20-n05


4 PDFs created and saved in /Users/jeremy/Desktop/_temporary_pdfs/_temp_KJA_RG-D_1a-21_1963-01_v20-n05
/Users/jeremy/Desktop/_temporary_pdfs/_temp_KJA_RG-D_1a-21_1963-01_v20-n05/KJA_RG-D_1a-21_1963-01_v20-n05.pdf created with size 0.04 GB

*****
KJA_RG-D_1a-21_1963-01_v20-n05 is now /Users/jeremy/Pictures/KJA/Output/1963/2.OCRed/KJA_RG-D_1a-21_1963-01_v20-n05
*****

4 *.tif to process in /Users/jeremy/Pictures/KJA/Output/1963/1.toOCR/KJA_RG-D_1a-21_1963-02_v20-n06


4 PDFs created and saved in /Users/jeremy/Desktop/_temporary_pdfs/_temp_KJA_RG-D_1a-21_1963-02_v20-n06
/Users/jeremy/Desktop/_temporary_pdfs/_temp_KJA_RG-D_1a-21_1963-02_v20-n06/KJA_RG-D_1a-21_1963-02_v20-n06.pdf created with size 0.04 GB

*****
KJA_RG-D_1a-21_1963-02_v20-n06 is now /Users/jeremy/Pictures/KJA/Output/1963/2.OCRed/KJA_RG-D_1a-21_1963-02_v20-n06
*****

4 *.tif to process in /Users/jeremy/Pictures/KJA/Output/1963/1.toOCR/KJA_RG-D_1a-21_1963-03_v20-n07


4 PDFs created and saved in /Users/jeremy/Desktop/_temporary_pdfs/_temp_KJA_RG-D_1a-21_1963-03_v20-n07
/Users/jeremy/Desktop/_temporary_pdfs/_temp_KJA_RG-D_1a-21_1963-03_v20-n07/KJA_RG-D_1a-21_1963-03_v20-n07.pdf created with size 0.04 GB

*****
KJA_RG-D_1a-21_1963-03_v20-n07 is now /Users/jeremy/Pictures/KJA/Output/1963/2.OCRed/KJA_RG-D_1a-21_1963-03_v20-n07
*****

4 *.tif to process in /Users/jeremy/Pictures/KJA/Output/1963/1.toOCR/KJA_RG-D_1a-21_1963-04_v20-n08


4 PDFs created and saved in /Users/jeremy/Desktop/_temporary_pdfs/_temp_KJA_RG-D_1a-21_1963-04_v20-n08
/Users/jeremy/Desktop/_temporary_pdfs/_temp_KJA_RG-D_1a-21_1963-04_v20-n08/KJA_RG-D_1a-21_1963-04_v20-n08.pdf created with size 0.04 GB

*****
KJA_RG-D_1a-21_1963-04_v20-n08 is now /Users/jeremy/Pictures/KJA/Output/1963/2.OCRed/KJA_RG-D_1a-21_1963-04_v20-n08
*****

4 *.tif to process in /Users/jeremy/Pictures/KJA/Output/1963/1.toOCR/KJA_RG-D_1a-21_1963-05_v20-n09


...

4 PDFs created and saved in /Users/jeremy/Desktop/_temporary_pdfs/_temp_KJA_RG-D_1a-21_1963-06_v20-n10
/Users/jeremy/Desktop/_temporary_pdfs/_temp_KJA_RG-D_1a-21_1963-06_v20-n10/KJA_RG-D_1a-21_1963-06_v20-n10.pdf created with size 0.04 GB

*****
KJA_RG-D_1a-21_1963-06_v20-n10 is now /Users/jeremy/Pictures/KJA/Output/1963/2.OCRed/KJA_RG-D_1a-21_1963-06_v20-n10
*****

16 *.tif to process in /Users/jeremy/Pictures/KJA/Output/1963/1.toOCR/KJA_RG-D_1a-21_1963-09_v21-n01


16 PDFs created and saved in /Users/jeremy/Desktop/_temporary_pdfs/_temp_KJA_RG-D_1a-21_1963-09_v21-n01
/Users/jeremy/Desktop/_temporary_pdfs/_temp_KJA_RG-D_1a-21_1963-09_v21-n01/KJA_RG-D_1a-21_1963-09_v21-n01.pdf created with size 0.15 GB

*****
KJA_RG-D_1a-21_1963-09_v21-n01 is now /Users/jeremy/Pictures/KJA/Output/1963/2.OCRed/KJA_RG-D_1a-21_1963-09_v21-n01
*****

4 *.tif to process in /Users/jeremy/Pictures/KJA/Output/1963/1.toOCR/KJA_RG-D_1a-21_1963-10_v21-n02


4 PDFs created and saved in /Users/jeremy/Desktop/_temporary_pdfs/_temp_KJA_RG-D_1a-21_1963-10_v21-n02
/Users/jeremy/Desktop/_temporary_pdfs/_temp_KJA_RG-D_1a-21_1963-10_v21-n02/KJA_RG-D_1a-21_1963-10_v21-n02.pdf created with size 0.04 GB

*****
KJA_RG-D_1a-21_1963-10_v21-n02 is now /Users/jeremy/Pictures/KJA/Output/1963/2.OCRed/KJA_RG-D_1a-21_1963-10_v21-n02
*****

4 *.tif to process in /Users/jeremy/Pictures/KJA/Output/1963/1.toOCR/KJA_RG-D_1a-21_1963-11_v21-n03


4 PDFs created and saved in /Users/jeremy/Desktop/_temporary_pdfs/_temp_KJA_RG-D_1a-21_1963-11_v21-n03
/Users/jeremy/Desktop/_temporary_pdfs/_temp_KJA_RG-D_1a-21_1963-11_v21-n03/KJA_RG-D_1a-21_1963-11_v21-n03.pdf created with size 0.04 GB

*****
KJA_RG-D_1a-21_1963-11_v21-n03 is now /Users/jeremy/Pictures/KJA/Output/1963/2.OCRed/KJA_RG-D_1a-21_1963-11_v21-n03
*****

4 *.tif to process in /Users/jeremy/Pictures/KJA/Output/1963/1.toOCR/KJA_RG-D_1a-21_1963-12_v21-n04


4 PDFs created and saved in /Users/jeremy/Desktop/_temporary_pdfs/_temp_KJA_RG-D_1a-21_1963-12_v21-n04
/Users/jeremy/Desktop/_temporary_pdfs/_temp_KJA_RG-D_1a-21_1963-12_v21-n04/KJA_RG-D_1a-21_1963-12_v21-n04.pdf created with size 0.04 GB

*****
KJA_RG-D_1a-21_1963-12_v21-n04 is now /Users/jeremy/Pictures/KJA/Output/1963/2.OCRed/KJA_RG-D_1a-21_1963-12_v21-n04
*****



In [4]:
# delete all '_temporary_pdfs_*' directories on the desktop
all_temp_dir_paths = sorted(Path('/Users/dlisla/Desktop/').glob('_temporary_pdfs_*'))
number_of_dir_paths = len(all_temp_dir_paths)
for temp_dir_path in all_temp_dir_paths:
    shutil.rmtree(temp_dir_path)

all_temp_dir_paths = sorted(Path('/Users/dlisla/Desktop/').glob('_temporary_pdfs_*'))
number_of_deleted_dir_paths = number_of_dir_paths - len(all_temp_dir_paths)

print(f'{number_of_deleted_dir_paths} _temporary_pdf_* directories deleted')

121 _temporary_pdf_* directories deleted
